In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()
# Pkg.upgrade_manifest()

In [ ]:
# using Cropbox
using Test
# using Plots
using TimeZones
using DataFramesMeta
using CSV
# Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
include("../src/Garlic.jl")

import Dates

obs_veget = CSV.read("$(@__DIR__)/../data/Phenology_Vegetative_2nd.csv", DataFrame);
obs_repro = CSV.read("$(@__DIR__)/../data/Phenology_Reproductive_2nd.csv", DataFrame);

In [ ]:
# Cropbox.hierarchy(Garlic.Context)

In [ ]:
tz = tz"Asia/Seoul"

Cucumber = @config (
    :Phenology => (
        optimal_temperature = 25, # Topt
        ceiling_temperature = 30, # Tceil
        critical_photoperiod = 12, # critPPD
        # initial_leaves_at_harvest = 5, # ILN
        maximum_leaf_initiation_rate = 1.70878, # LIR
        maximum_emergence_rate = 0.2,
        # maximum_leaf_initiation_rate = 0.1003, # LIR
        maximum_phyllochron_asymptote = 1.75561, # LTARa
        leaves_generic = 50, # GLN
    ),
    :Leaf => (
        maximum_elongation_rate = 4.70, # LER
        minimum_length_of_longest_leaf = 25.0, # LL
        # stay_green = 1.84, # SG
        stay_green = 20,
        length_to_width_ratio = 1,
        leaf_detaching_rate = 23, #LDR
    ),
    :Carbon => (
        maintenance_respiration_coefficient = 0.012, # Rm
        synthesis_efficiency = 0.8, # Yg
    ),
    :Meta => (; cultivar = :Cucumber),
    :Plant => (initial_planting_density = 55,),
);

In [ ]:
Calibrate = @config(Cucumber, (
    :Phenology => (
        planting_date = ZonedDateTime(2020, 10, 22, tz"Asia/Seoul"),
        emergence_date = ZonedDateTime(2020, 10, 23, tz"Asia/Seoul"),
        first_flowering_date = ZonedDateTime(2020, 11, 19, tz"Asia/Seoul"),
    ),
    :Meta => (
        planting_group = 1,
        year = 2020,
    ),
    :Calendar => (
        init = ZonedDateTime(2020, 10, 22, tz"Asia/Seoul"),
        last = ZonedDateTime(2021, 2, 15, tz"Asia/Seoul"),
    ),
    :Weather => (
        CO2 = 500, # CO2 Enrichment
        store = Garlic.loadwea("$(@__DIR__)/../data/Cucumber1st.wea", tz"Asia/Seoul"),
    ),
));


In [ ]:
Validate = @config(Cucumber, (
    :Phenology => (
        planting_date = ZonedDateTime(2021, 2, 1, tz"Asia/Seoul"), # Y1 sow
        emergence_date = ZonedDateTime(2021, 2, 23, tz"Asia/Seoul"), # Y1 emg
        first_flowering_date = ZonedDateTime(2021, 3, 24, tz"Asia/Seoul"),
    ),
    :Meta => (
        planting_group = 2,
        year = 2021,
    ),
    :Calendar => (
        init = ZonedDateTime(2021, 2, 1, tz"Asia/Seoul"),
        last = ZonedDateTime(2021, 8, 29, tz"Asia/Seoul"),
    ),
    :Weather => (
        CO2 = 500, # CO2 Enrichment
        store = Garlic.loadwea("$(@__DIR__)/../data/Cucumber2nd.wea", tz"Asia/Seoul"),
    ),
));

In [ ]:
# obs_veget_unit = obs_veget |> unitfy
# obs_repro_unit = obs_repro |> unitfy

# f(s) = s.DAP' in obs_repro_unit.DAP && Dates.hour(s.calendar.time') == 12

# calibrate(Garlic.Model, obs_repro_unit;
#     config=Cucumber,
#     stop="calendar.count",
#     index=:DAP,
#     target=:flowers_appeared => :flowers_appeared,
#     parameters= :Phenology => (;
#         FAR_max = (0.4, 1.8),
#         FIR_max = (0.4, 1.8),
#     ),
#     snap=f,
#     optim=(:MaxSteps => 20,),
# )

In [ ]:
# obs_veget_unit = obs_veget |> unitfy
# obs_repro_unit = obs_repro |> unitfy

# f(s) = s.DAP' in obs_repro_unit.DAP && Dates.hour(s.calendar.time') == 12

# calibrate(Garlic.Model, obs_repro_unit;
#     config=Cucumber,
#     stop="calendar.count",
#     index=:DAP,
#     target=:fruits_appeared => :fruits_appeared,
#     parameters= :Phenology => (;
#         FRIR_max = (0.4, 1.8),
#         FRAR_max = (0.4, 1.8),
#     ),
#     snap=f,
#     optim=(:MaxSteps => 20,),
# )

In [ ]:
# obs_veget_unit = obs_veget |> unitfy
# obs_repro_unit = obs_repro |> unitfy

# f(s) = s.DAP' in obs_veget_unit.DAP && Dates.hour(s.calendar.time') == 12

# calibrate(Garlic.Model, obs_veget_unit;
#     config=Cucumber,
#     stop="calendar.count",
#     index=:DAP,
#     target=:internode_count => :leaves_appeared,
#     parameters= :Phenology => (;
#         LTARa_max = (0.4, 1.8),
#         LIR_max = (0.4, 1.8),
#     ),
#     snap=f,
#     optim=(:MaxSteps => 50,),
# )

In [ ]:
obs_veget_unit = obs_veget |> unitfy
obs_repro_unit = obs_repro |> unitfy

r = simulate(Garlic.Model;
    config=Validate,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
)
# @test r.leaves_initiated[end] > 0

plot_veg = visualize(r, :DAP, [:leaves_appeared, :leaves_real], kind=:line)
visualize!(plot_veg, obs_veget_unit, :DAP, [:leaf_count, :internode_count]) |> display # Fig. 3.D
plot_rep = visualize(r, :DAP, [:flowers_appeared], kind=:line)
visualize!(plot_rep, obs_repro_unit, :DAP, [:flowers_appeared]) |> display
plot_rep_f = visualize(r, :DAP, [:fruits_appeared], kind=:line)
visualize!(plot_rep_f, obs_repro_unit, :DAP, [:fruits_appeared]) |> display
visualize(r, :DAP, :LAI, kind=:line) |> display # Fig. 4.D
visualize(r, :DAP, [:leaf_mass, :total_mass, :fruit_mass]) |> display
visualize(r, :DAP, [:A_net], kind=:line) |> display
# p = visualize(r, :DAP, [:leaves_real], kind=:line)

In [ ]:
evaluate(Garlic.Model, obs_veget_unit; 
    config = Validate,
    index = :DAP => :DAP,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
    target = :leaf_count => :leaves_real, metric = :ef)

In [ ]:
evaluate(Garlic.Model, obs_repro_unit; 
    config = Validate,
    index = :DAP => :DAP,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
    target = :flowers_appeared => :flowers_appeared, metric = :ef)


In [ ]:
evaluate(Garlic.Model, obs_repro_unit; 
    config = Validate,
    index = :DAP => :DAP,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
    target = :fruits_appeared => :fruits_appeared, metric = :ef)


In [ ]:
# manipulate(Garlic.Model, :DAP, [:leaves_appeared, :leaves_mature, :leaves_dropped, :leaves_detached, :leaves_real];
#     config = Cucumber,
#     parameters = :Phenology => (;
#         optimal_temperature = 10:40,
        
#     ),
#     stop = "calendar.count",
#     kind = :line,
# )